Before you submit this notebook, make sure everything runs as expected in the local test cases. 
Please, paste the solution to the designed cell and do not change anything else.

Also, please, leave your first and last names below

In [1]:
FirstName = "Viacheslav"
LastName = "Barinov"

---

In [2]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.preprocessing import OneHotEncoder

In [3]:
def entropy(y):  
    EPS = 0.0005
    proba = np.sum(y, axis = 0)/y.shape[0]
    log_proba = np.log(proba + EPS)

    res =  -np.dot(proba, log_proba.T)
    
    return res
    
def gini(y):
    proba = np.sum(y, axis = 0)/y.shape[0]
    res = 1 - np.sum(np.power(proba,2))
    
    return res
    
def variance(y):
    y = np.power(y - np.mean(y), 2)
    res = np.sum(y)/y.shape[0]
    return res

def mad_median(y):
    y = np.abs(y - np.median(y))
    res = np.sum(y)/y.shape[0]
    return res


def one_hot_encode(n_classes, y):
    y_one_hot = np.zeros((len(y), n_classes), dtype=float)
    y_one_hot[np.arange(len(y)), y.astype(int)[:, 0]] = 1.
    return y_one_hot


def one_hot_decode(y_one_hot):
    return y_one_hot.argmax(axis=1)[:, None]


class Node:
    def __init__(self, feature_index, threshold, proba=0):
        self.feature_index = feature_index
        self.value = threshold
        self.proba = proba
        self.left_child = None
        self.right_child = None
        
        
class DecisionTree(BaseEstimator):
    all_criterions = {
        'gini': (gini, True),
        'entropy': (entropy, True),
        'variance': (variance, False),
        'mad_median': (mad_median, False)
    }

    def __init__(self, n_classes=None, max_depth=np.inf, min_samples_split=2, 
                 criterion_name='gini', debug=False):

        assert criterion_name in self.all_criterions.keys(), 'Criterion name must be on of the following: {}'.format(self.all_criterions.keys())
        
        self.n_classes = n_classes
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion_name = criterion_name
        self.depth = 0
        self.root = None
        self.debug = debug
        
        
        
        
        
    def make_split(self, feature_index, threshold, X_subset, y_subset):
        left = X_subset[:, feature_index] < threshold
        right = X_subset[:, feature_index] >= threshold
        

        X_left = X_subset[left, : ]
        y_left = y_subset[left, : ]

        X_right = X_subset[right, : ]
        y_right= y_subset[right, : ]

        return (X_left, y_left), (X_right, y_right)
    
    def make_split_only_y(self, feature_index, threshold, X_subset, y_subset):
        left = X_subset[:, feature_index] < threshold
        right = X_subset[:, feature_index] >= threshold

        y_left = y_subset[left, : ]

        y_right= y_subset[right, : ]
        
        return y_left, y_right

    def choose_best_split(self, X_subset, y_subset):
        H_m = self.criterion(y_subset)
        feature_index, threshold, best_result = -1, -1, -1e9
        for i in range(X_subset.shape[1]):
            values = np.sort(np.unique(X_subset[:, i]))
            for j in range(1, len(values)):
                left, right = self.make_split_only_y(i,values[j], X_subset, y_subset)
                H_l_n = self.criterion(left) * left.shape[0] / y_subset.shape[0]
                H_r_n = self.criterion(right) * right.shape[0] / y_subset.shape[0]
                res = H_m - H_l_n - H_r_n
                if res > best_result:
                    feature_index = i
                    threshold = values[j]
                    best_result = res

            

        return feature_index, threshold
    
    def make_tree(self, X_subset, y_subset, depth = 0):
        new_node = Node(-1, -1, proba=np.sum(y_subset,axis = 0) / y_subset.shape[0])
        if depth >= self.max_depth or X_subset.shape[0] < self.min_samples_split:
            return new_node
        if self.classification and np.max(np.sum(y_subset, axis = 0)) == y_subset.shape[0]:
            return new_node
        if not self.classification and np.unique(y_subset).shape[0] == 1:
            return new_node
        feature_index, threshold = self.choose_best_split(X_subset, y_subset)
        if feature_index == -1:
            return new_node
        new_node.feature_index = feature_index
        new_node.value = threshold
        left, right = self.make_split(feature_index, threshold, X_subset, y_subset)
        new_node.left_child = self.make_tree(left[0], left[1], depth + 1)
        new_node.right_child = self.make_tree(right[0], right[1], depth + 1)
        return new_node

    
           
    def fit(self, X, y):
        assert len(y.shape) == 2 and len(y) == len(X), 'Wrong y shape'
        self.criterion, self.classification = self.all_criterions[self.criterion_name]
        if self.classification:
            if self.n_classes is None:
                self.n_classes = len(np.unique(y))
            y = one_hot_encode(self.n_classes, y)


        self.root = self.make_tree(X, y)
    def predict_help(self, X, node: Node, mask = None):
        if mask is None:
            y = [node.proba for i in range(X.shape[0])]
            y = np.array(y)
            mask = np.full(X.shape[0], True)
        else:
            y = np.zeros((X.shape[0], node.proba.shape[0]))
            y[mask, : ] = node.proba
        if not node.left_child  is None:
            mask_l = np.logical_and(mask, X[:, node.feature_index] < node.value)
            y[mask_l] = self.predict_help(X, node.left_child, mask_l)
        if not node.right_child  is None:
            mask_r = np.logical_and(mask, np.logical_not(mask_l))
            y[mask_r] = self.predict_help(X, node.right_child, mask_r)
        return y[mask]

    def predict(self, X):
        y_predicted = self.predict_help(X, self.root, None)
        if self.classification:
            y_predicted = np.argmax(y_predicted, axis=1)
        return y_predicted
        
    def predict_proba(self, X):
        assert self.classification, 'Available only for classification problem'
        y_predicted_probs = self.predict_help(X, self.root)

        return y_predicted_probs

### Test 0: Initialization (0.01 points)

In [4]:
# do not change this cell
import numpy as np
import unittest
import sys
import io

from sklearn.datasets import fetch_california_housing, load_digits
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

digits_data = load_digits(n_class=2).data
digits_target = load_digits(n_class=2).target[:, None]



### Test 1: Make splits loops (0.1 points)

In [5]:
X = np.ones((4, 5), dtype=float) * np.arange(4)[:, None]
y = np.arange(4)[:, None] + np.asarray([0.2, -0.3, 0.1, 0.4])[:, None]
class_estimator = DecisionTree(max_depth=5, criterion_name='gini')

(X_l, y_l), (X_r, y_r) = class_estimator.make_split(1, 1., X, y)

flag_X = np.array_equal(X[:1], X_l) and np.array_equal(X[1:], X_r) 
flag_y = np.array_equal(y[:1], y_l) and np.array_equal(y[1:], y_r)
assert flag_X and flag_y

### Test 2: Gini accuracy (0.2 points)

In [6]:
class_estimator = DecisionTree(max_depth=5, criterion_name='gini')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
accuracy_gini = accuracy_score(y_test, ans)
assert 0.96 < accuracy_gini

NameError: name 'X_train' is not defined

### Test 3: Entropy accuracy (0.2 points)

In [ ]:
class_estimator = DecisionTree(max_depth=5, criterion_name='entropy')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
accuracy = accuracy_score(y_test, ans)
assert 0.96 < accuracy

### Test 4: Entropy probabilities (0.2 points)

In [ ]:
class_estimator = DecisionTree(max_depth=10, criterion_name='entropy')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
reference = np.array([0.48611111, 0.51388889])
assert np.abs(np.sum(class_estimator.predict_proba(X_test).mean(axis=0) - reference)) < 1e-6

### Test 5: MSE mad_median (0.15 points)

In [ ]:
housing = fetch_california_housing()
random_indices = np.random.choice(np.arange(len(housing.data)), 500)

regr_data = housing.data[random_indices]
regr_target = housing.target[random_indices, None]
RX_train, RX_test, Ry_train, Ry_test = train_test_split(regr_data, regr_target, test_size=0.2, random_state=42)

regressor = DecisionTree(max_depth=8, criterion_name='mad_median')
regressor.fit(RX_train, Ry_train)
predictions_mad = regressor.predict(RX_test)
mse = mean_squared_error(Ry_test, predictions_mad)
assert 0.4 < mse < 2

### Test 6: MSE Variance (0.15 points)

In [ ]:
housing = fetch_california_housing()
random_indices = np.random.choice(np.arange(len(housing.data)), 500)

regr_data = housing.data[random_indices]
regr_target = housing.target[random_indices, None]
RX_train, RX_test, Ry_train, Ry_test = train_test_split(regr_data, regr_target, test_size=0.2, random_state=42)

regressor = DecisionTree(max_depth=8, criterion_name='variance')
regressor.fit(RX_train, Ry_train)
predictions_mad = regressor.predict(RX_test)
mse = mean_squared_error(Ry_test, predictions_mad)
assert 0.5 < mse < 1.8